<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install workalendar
!pip install pytz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.0 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732000 sha256=cecdad2c224746680bbe8418403b5b8e40f9b400acd74d6d918216545e5d77b7
  Stored in directory: /root/.cache/pip/wheels/8f/bd/f9/5c4c39b529e0322b08979e1c465e203218bc2cca75d20f7df5
Successfully built pymeeus


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Extraer Canales Realizados y crear Hoja REGISTRO REALIZADA**

# Teléfono Realizada

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelefono = pd.read_excel(file_path, sheet_name='Teléfono')


dftelefono.head()

,EMPRESA,REALIZADA,FECHA
0,ACER,REALIZADO,2025-05-06
1,ACER,REALIZADO,2025-05-06
2,ACER,REALIZADO,2025-05-06
3,ACER,REALIZADO,2025-05-06
4,ACER,REALIZADO,2025-05-07


**Contar previamente la cantidad de realizado presentes**

In [ ]:
# Eliminar espacios antes y después en los nombres de las columnas
dftelefono.columns = dftelefono.columns.str.strip()

# Eliminar espacios antes y después en todo el contenido tipo string
dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)

conteo = dftelefono['REALIZADA'].str.strip().str.lower()
total_REALIZADO = (conteo == 'realizado').sum()
print(f"Total de 'REALIZADO': {total_REALIZADO}")


Total de 'REALIZADO': 3829


/tmp/ipython-input-4-3075620810.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dftelefono = dftelefono.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].str.strip()

dftelefono['EMPRESA'] = (dftelefono['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Corregir las empresas que están mal escritas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].replace(reemplazos)

dftelefono['EMPRESA']
print(dftelefono[dftelefono['EMPRESA'] == 'canal de isabel ii'])


                 EMPRESA  REALIZADA      FECHA
320   canal de isabel ii  REALIZADO 2025-05-06
321   canal de isabel ii  REALIZADO 2025-05-06
322   canal de isabel ii  REALIZADO 2025-05-06
323   canal de isabel ii  REALIZADO 2025-05-07
324   canal de isabel ii  REALIZADO 2025-05-07
325   canal de isabel ii  REALIZADO 2025-05-07
326   canal de isabel ii  REALIZADO 2025-05-07
327   canal de isabel ii  REALIZADO 2025-05-08
328   canal de isabel ii  REALIZADO 2025-05-08
329   canal de isabel ii  REALIZADO 2025-05-08
330   canal de isabel ii  REALIZADO 2025-05-08
331   canal de isabel ii  REALIZADO 2025-05-09
332   canal de isabel ii  REALIZADO 2025-05-09
333   canal de isabel ii  REALIZADO 2025-05-09
334   canal de isabel ii  REALIZADO 2025-05-12
335   canal de isabel ii  REALIZADO 2025-05-12
336   canal de isabel ii  REALIZADO 2025-05-12
337   canal de isabel ii  REALIZADO 2025-05-12
338   canal de isabel ii  REALIZADO 2025-05-13
339   canal de isabel ii  REALIZADO 2025-05-13
340   canal d

**Generar Hoja Registro para Realizada Teléfono**

In [ ]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
dftelefono = dftelefono.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Estandarizar columna 'REALIZADA'
dftelefono['realizada_limpia'] = dftelefono['REALIZADA'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dftelefono[dftelefono['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Telefono'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dftelefono = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dftelefono = dftelefono[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dftelefono.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dftelefono


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Realizada,76
1,adeslas,Telefono,Realizada,22
2,adt,Telefono,Realizada,69
3,aeda homes,Telefono,Realizada,39
4,aegon salud,Telefono,Realizada,49
...,...,...,...,...
95,volotea,Telefono,Realizada,38
96,vueling,Telefono,Realizada,44
97,whirlpool,Telefono,Realizada,23
98,xiaomi,Telefono,Realizada,25


In [ ]:
total_cantidad = dftelefono['cantidad'].sum()
print("Total de la columna 'cantidad':", total_cantidad)


Total de la columna 'cantidad': 3829


# Mail Realizada

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfmail = pd.read_excel(file_path, sheet_name='Mail')


dfmail.head()

,EMPRESA,ESTADO CORREO,FECHA
0,ACER,REALIZADO,2025-05-07
1,ACER,REALIZADO,2025-05-07
2,AEDA HOMES,REALIZADO,2025-05-07
3,AGBAR,REALIZADO,2025-05-07
4,AGBAR,REALIZADO,2025-05-07


**Contar la cantidad de enviado**

In [ ]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfmail['ESTADO CORREO'].str.strip().str.lower() == 'realizado').sum()
print(f"Cantidad de 'realizado': {enviados}")

Cantidad de 'realizado': 1450


/tmp/ipython-input-10-1451721913.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfmail = dfmail.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Corregir las empresas que están mal escritas**

In [ ]:
#Pasar a minúscula
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfmail['EMPRESA'] = dfmail['EMPRESA'].replace(reemplazos)

dfmail['EMPRESA']

,EMPRESA
0,acer
1,acer
2,aeda homes
3,agbar
4,agbar
...,...
1445,general óptica
1446,lg
1447,lg tv
1448,norauto


**Pasar todos los datos de Empresa a minúscula**

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.strip()

dfmail['EMPRESA'] = (dfmail['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar hoja Registro de Realizadas Mail**

In [ ]:
# Estandarizar columna 'REALIZADA'
dfmail['realizada_limpia'] = dfmail['ESTADO CORREO'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfmail[dfmail['realizada_limpia'].isin(['realizado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Mail/formulario'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfmail = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfmail = dfmail[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfmail.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfmail.head()

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Mail/formulario,Realizada,26
1,adeslas,Mail/formulario,Realizada,12
2,adt,Mail/formulario,Realizada,28
3,aeda homes,Mail/formulario,Realizada,15
4,aegon salud,Mail/formulario,Realizada,8


# WEB Realizada

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfweb = pd.read_excel(file_path, sheet_name='Web')


dfweb.head()

,EMPRESA,ESTADO NAVEGACIÓN,FECHA
0,ACER,ENCONTRADA,2025-05-13
1,ACER,ENCONTRADA,2025-05-14
2,ACER,ENCONTRADA,2025-05-15
3,ACER,ENCONTRADA,2025-05-16
4,ADESLAS,ENCONTRADA,2025-05-13


In [ ]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)

enviados = (dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower() == 'encontrada').sum()
print(f"Cantidad de 'encontrada': {enviados}")

Cantidad de 'encontrada': 54


/tmp/ipython-input-15-2137601728.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfweb = dfweb.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfweb['EMPRESA'] = dfweb['EMPRESA'].str.strip()

dfweb['EMPRESA'] = (dfweb['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Corregir las empresas que están mal escritas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfweb['EMPRESA'] = dfweb['EMPRESA'].replace(reemplazos)

dfweb['EMPRESA']

,EMPRESA
0,acer
1,acer
2,acer
3,acer
4,adeslas
5,adeslas
6,aeda homes
7,aeda homes
8,aeda homes
9,asisa


In [ ]:
# Estandarizar columna 'REALIZADA'
dfweb['realizada_limpia'] = dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfweb[dfweb['realizada_limpia'].isin(['encontrada'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Web'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfweb = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfweb = dfweb[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfweb.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfweb

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Web,Realizada,4
1,adeslas,Web,Realizada,2
2,aeda homes,Web,Realizada,3
3,asisa,Web,Realizada,2
4,asus,Web,Realizada,2
5,audika,Web,Realizada,2
6,aurgi,Web,Realizada,2
7,avis,Web,Realizada,2
8,beko,Web,Realizada,1
9,bosch,Web,Realizada,1


# Facebook y Twitter Realizada

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredes = pd.read_excel(file_path, sheet_name='Redes')


dfredes.head()

,EMPRESA,tipo_contacto,ESTADO RR SS,FECHA
0,ASUS,FACEBOOK,REALIZADO,2025-05-13
1,AURGI,FACEBOOK,REALIZADO,2025-05-13
2,AVIS,FACEBOOK,REALIZADO,2025-05-13
3,BEKO,FACEBOOK,REALIZADO,2025-05-13
4,BOSCH,FACEBOOK,REALIZADO,2025-05-13


**Corregir las empresas que están mal escritas**

In [ ]:
#Pasar a minúscula
dfredes['EMPRESA'] = dfredes['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfredes['EMPRESA'] = dfredes['EMPRESA'].replace(reemplazos)

dfredes['EMPRESA']

,EMPRESA
0,asus
1,aurgi
2,avis
3,beko
4,bosch
5,bosch talleres
6,bridgestone
7,canal de isabel ii
8,dia fidelización
9,continental


**Recuento de la cantidad por canal**

In [ ]:
# Eliminar espacios antes y después en todo el contenido tipo string
dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].astype(str).str.strip().str.lower()
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].fillna('').str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo los "enviado"
df_enviados = dfredes[dfredes['estado_rr_ss_limpio'] == 'realizado']

# Verificamos si el DataFrame tiene datos antes de proceder
if not df_enviados.empty:
    conteo = df_enviados['tipo_contacto'].value_counts().loc[['Facebook', 'Twitter']].to_dict()
    print(f"Cantidad de 'enviado': {conteo}")
else:
    print("No hay datos disponibles en el DataFrame para procesar.")

# Mostramos resultado
print(f"Cantidad de 'enviado': {conteo}")


Cantidad de 'enviado': {'Facebook': 40, 'Twitter': 6}
Cantidad de 'enviado': {'Facebook': 40, 'Twitter': 6}


/tmp/ipython-input-21-676816704.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfredes = dfredes.applymap(lambda x: x.strip() if isinstance(x, str) else x)


**Ejecutar para Facebook**

In [ ]:
# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()  # o 'tipo_contacto' si es otro nombre

# Filtramos solo "enviado" por canal
df_facebook = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebook]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_facebook = df_facebook.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebook.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebook.head()


📘 Facebook:


/tmp/ipython-input-22-4248538293.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
/tmp/ipython-input-22-4248538293.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Facebook,Realizada,2
1,aurgi,Facebook,Realizada,2
2,avis,Facebook,Realizada,4
3,beko,Facebook,Realizada,4
4,bosch,Facebook,Realizada,1


In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_facebook['empresa'] = df_facebook['empresa'].str.strip()

df_facebook['empresa'] = (df_facebook['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Ejecutar para Twitter**

In [ ]:
# Filtramos solo "enviado" por canal
df_twitter  = dfredes[(dfredes['estado_rr_ss_limpio'] == 'realizado') & (dfredes['tipo_contacto'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitter]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_twitter  = df_twitter.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitter.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

print("\n Twitter:")
df_twitter.head()


 Twitter:


/tmp/ipython-input-24-1201765053.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Realizada'
/tmp/ipython-input-24-1201765053.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['tipo_contacto']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Twitter,Realizada,2
1,aurgi,Twitter,Realizada,2
2,bosch,Twitter,Realizada,1
3,dia fidelización,Twitter,Realizada,1


In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_twitter['empresa'] = df_twitter['empresa'].str.strip()

df_twitter['empresa'] = (df_twitter['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Chat y WhatsApp Realizada

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchats = pd.read_excel(file_path, sheet_name='Chats')


dfchats.head()

,EMPRESA,tipo_contacto,ESTADO CHAT,FECHA
0,ACER,CHAT WEB,REALIZADO,2025-05-13
1,ASUS,CHAT WEB,REALIZADO,2025-05-13
2,BOSCH,CHAT WEB,REALIZADO,2025-05-13
3,BRUNEAU,CHAT WEB,REALIZADO,2025-05-13
4,DINAHOSTING,CHAT WEB,REALIZADO,2025-05-13


**Corregir las empresas que están mal escritas**

In [ ]:
#Pasar a minúscula
dfchats['EMPRESA'] = dfchats['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfchats['EMPRESA'] = dfchats['EMPRESA'].replace(reemplazos)

dfchats['EMPRESA']

,EMPRESA
0,acer
1,asus
2,bosch
3,bruneau
4,dinahosting
5,edenred
6,acer
7,asus
8,bosch
9,bruneau


In [ ]:
# --- PASO 1: Asegurar minúsculas en columnas relevantes ---
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.lower()
dfchats['EMPRESA'] = (dfchats['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

# --- PASO 2: Definir palabras clave válidas y reemplazos ---
servicios_validos = [
    'twitter','facebook', 'canal social media facebook', 'social media facebook', 'chat virtual', 'chat',  'whatsapp', 'chat web'
]

reemplazos_servicio = {
    'teléfono': 'Telefono',
    'llamadas': 'Telefono',
    'telefónico': 'Telefono',
    'canal telefónico': 'Telefono',
    'canal web':'Web',
    'canal social media facebook':'Facebook',
    'social media facebook': 'Facebook',
    'mail/formulario':'Mail/formulario',
    'web': 'Web',
    'facebook':'Facebook',
    'twitter':'Twitter',
    'chat web': 'Chat',
    'whatsapp': 'WhatsApp',
    'webs':'Web',
    'chat virtual': 'Chat',
    'chat': 'Chat'
}

# --- PASO 3: Función para limpiar SERVICIO ---
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos_servicio.get(palabra, palabra)
    return None

# --- PASO 4: Aplicar limpieza servicio ---
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].apply(limpiar_servicio)

print(dfchats['tipo_contacto'])

0     Chat
1     Chat
2     Chat
3     Chat
4     Chat
5     Chat
6     Chat
7     Chat
8     Chat
9     Chat
10    Chat
11    Chat
12    Chat
13    Chat
14    Chat
15    Chat
16    Chat
17    Chat
18    Chat
19    Chat
20    Chat
21    Chat
22    Chat
23    Chat
Name: tipo_contacto, dtype: object


**Contabilizar la cantidad por canal de realizadas**

In [ ]:
# Estandarizamos texto ANTES de cualquier filtro
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.lower()

# Filtramos solo los "realizado"
df_enviados = dfchats[dfchats['estado_chat_limpio'] == 'realizado']

# Definimos categorías válidas
categorias_validas = ['whatsapp', 'chat']

print(df_enviados['tipo_contacto'].value_counts())


tipo_contacto
chat    24
Name: count, dtype: int64


**Crear hoja Registro para WhatsApp**

In [ ]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_wapp = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Whatsapp')]

# Agregar columnas fijas directamente a df_wapp
df_wapp['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_wapp = df_wapp.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_wapp.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_wapp['tipo_contacto'] = df_wapp['tipo_contacto'].replace('Whatsapp', 'WhatsApp')


# Mostrar resultado
print("📘 WhatsApp:")
df_wapp.head()

📘 WhatsApp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


Pasar las empresas a minúscula

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_wapp['empresa'] = df_wapp['empresa'].str.strip()

df_wapp['empresa'] = (df_wapp['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Crear hoja Chat Web para realizadas**

In [ ]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_chat = dfchats[(dfchats['estado_chat_limpio'] == 'realizado') & (dfchats['tipo_contacto'] == 'Chat')]

# Agregar columnas fijas directamente a df_wapp
df_chat['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_chat = df_chat.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_chat.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
#df_chat['tipo_contacto'] = df_chat['tipo_contacto'].replace('Chat', 'Chat web')

# Mostrar resultado
print("📘 Chat:")
df_chat.head()

📘 Chat:


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Chat,Realizada,4
1,asus,Chat,Realizada,4
2,bosch,Chat,Realizada,4
3,bruneau,Chat,Realizada,4
4,dinahosting,Chat,Realizada,4


Pasar nombre empresa a minúscula

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_chat['empresa'] = df_chat['empresa'].str.strip()

df_chat['empresa'] = (df_chat['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Agrupar todos los canales Realizada

In [ ]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_realizada = pd.concat([dftelefono, dfmail, df_facebook, df_twitter, df_chat, df_wapp, dfweb], ignore_index=True)
df_final_realizada = df_final_realizada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_realizada)

                empresa tipo_contacto tipo_de_acción  cantidad
0                  acer      Telefono      Realizada        76
1               adeslas      Telefono      Realizada        22
2                   adt      Telefono      Realizada        69
3            aeda homes      Telefono      Realizada        39
4           aegon salud      Telefono      Realizada        49
..                  ...           ...            ...       ...
235                 bsm           Web      Realizada         3
236  canal de isabel ii           Web      Realizada         4
237              cottet           Web      Realizada         6
238                ikea           Web      Realizada         4
239       sector alarma           Web      Realizada         6

[240 rows x 4 columns]


In [ ]:
recuento_contacto = df_final_realizada.groupby('tipo_contacto')['cantidad'].sum().reset_index()
print("Cantidad total por tipo de contacto:")
print(recuento_contacto)

total_general = df_final_realizada['cantidad'].sum()
print(f"Total general de cantidad: {total_general}")

Cantidad total por tipo de contacto:
     tipo_contacto  cantidad
0             Chat        24
1         Facebook        40
2  Mail/formulario      1450
3         Telefono      3829
4          Twitter         6
5              Web        54
Total general de cantidad: 5403


In [ ]:
# Guardar el DataFrame modificado como archivo Excel
df_final_realizada.to_excel('Realizadas.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Realizadas.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ---------------------------------------------------------------------------------------------------------------------------

# **Evaluadas**

# Teléfono

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelef = pd.read_excel(file_path, sheet_name='Teléfono')


dftelef.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C22. IMAGEN CORPORATIVA.COND.NO SE DISCULPA ANTE LAS MOLESTIAS EVIDENCIADAS POR EL CLIENTE DURANTE LA LLAMADA.,C22. IMAGEN CORPORATIVA.OBS,C23. DESPEDIDA,"C23. DESPEDIDA.COND.EL ASESOR SE DESPIDE DEL CLIENTE CON SU NOMBRE, HABIÉNDOSE ASEGURADO DE QUE TODAS SUS DUDAS ESTÉN RESUELTAS.","C23. DESPEDIDA.COND.EL ASESOR SE DESPIDE DEL CLIENTE CON SU NOMBRE, PERO NO SE ASEGURA QUE LE QUEDEN DUDAS POR RESOLVER.","C23. DESPEDIDA.COND.EL ASESOR SE ASEGURA QUE TODAS LAS DUDAS DEL CLIENTE ESTÉN RESUELTAS, PERO NO UTILIZA EL NOMBRE DE ESTE EN LA DESPEDIDA.",C23. DESPEDIDA.COND.EL ASESOR NI SE DESPIDE DEL CLIENTE CON SU NOMBRE NI SE ASEGURA QUE LE QUEDEN DUDAS POR RESOLVER.,C23. DESPEDIDA.COND.NO HA HABIDO DESPEDIDA AL NO HABERSE TRANSFERIDO LA LLAMADA.,C23. DESPEDIDA.COND.NO HA HABIDO DESPEDIDA AL CORTARSE LA TRANSFERENCIA.,C23. DESPEDIDA.OBS
0,101S_1013-OP_11_100_20250506H_15_12,01_HERTZ_01,HERTZ 01,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-14,...,NaN,NaN,1.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN
1,101S_1013-OP_11_100_20250507H_12_28,02_HERTZ_02,HERTZ 02,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-14,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_1013-OP_11_100_20250514H_13_19,07_HERTZ_05,HERTZ 05,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-30,...,NaN,NaN,1.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN
3,101S_1013-OP_11_100_20250515H_10_16,08_HERTZ_07,HERTZ 07,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-30,...,NaN,NaN,1.0,NaN,0.0,1.0,NaN,NaN,NaN,NaN
4,101S_1013-OP_11_100_20250519H_11_05,09_HERTZ_10,HERTZ 10,NaN,1013,Hertz Teléfono,Hertz,NaN,10057,2025-05-30,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


**Limpia los nombres de la columna**

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelef['EMPRESA'] = dftelef['EMPRESA'].str.lower().str.strip()

#ftelef['EMPRESA'] = (dftelef['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Reemplazo de las empresas que están mal escritas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    '1 and 1 IONOS': 'ionos',
    ' and  ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'

}

# Aplica los reemplazos
dftelef['EMPRESA'] = dftelef['EMPRESA'].replace(reemplazos)


**Armar el archivo de resumen**

In [ ]:
# Asegurar que la columna se llama 'empresa'
dftelef.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dftelef['tipo_contacto'] = 'Telefono'
dftelef['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dftelef = dftelef.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dftelef = dftelef[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dftelef

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Evaluada,48
1,adeslas,Telefono,Evaluada,9
2,adt,Telefono,Evaluada,28
3,aeda homes,Telefono,Evaluada,35
4,aegon salud,Telefono,Evaluada,16
...,...,...,...,...
80,volotea,Telefono,Evaluada,10
81,vueling,Telefono,Evaluada,30
82,whirlpool,Telefono,Evaluada,17
83,xiaomi,Telefono,Evaluada,17


# Mail y Formulario

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfemail = pd.read_excel(file_path, sheet_name='Mail')


dfemail.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C18. PROACTIVIDAD.COND.TOMA DECISIONES EFICIENTES Y EFICACES PARA RESOLVER LA CONSULTA.,"C18. PROACTIVIDAD.COND.EXCESO DE INFORMACIÓN NO ÚTIL, QUE NO APORTA VALOR AL CLIENTE.",C18. PROACTIVIDAD.COND.NO ADAPTA LOS PROCESOS EN LA MEDIDA DE LO POSIBLE Y SÓLO OFRECE ALTERNATIVAS POR INSISTENCIA DEL CLIENTE.,C18. PROACTIVIDAD.COND.NO TOMA DECISIONES EFICIENTES Y EFICACES PARA RESOLVER LA CONSULTA,C18. PROACTIVIDAD.OBS,C19. SEGUIMIENTO DE LA RELACIÓN,"C19. SEGUIMIENTO DE LA RELACIÓN.COND.EL ASESOR DEJA UNO O VARIOS CANALES ALTERNATIVOS DE CONTACTO (TELÉFONO, RR SS Y CHATS).",C19. SEGUIMIENTO DE LA RELACIÓN.COND.EL ASESOR NO DEJA NINGÚN CANAL ALTERNATIVO DE CONTACTO.,C19. SEGUIMIENTO DE LA RELACIÓN.COND.EL ASESOR SÓLO DEJA EL CANAL WEB O EL MISMO CORREO ELECTRÓNICO COMO CANAL ALTERNATIVO DE CONTACTO.,C19. SEGUIMIENTO DE LA RELACIÓN.OBS
0,101S_1014-OP_11_100_20250508H_16_42,01_HERTZ_01,HERTZ 01,NaN,1014,Hertz Mail,Hertz,NaN,10058,2025-06-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101S_1026-OP_11_100_20250507H_16_48,01_METROVACESA_01,METROVACESA 01,NaN,1026,Metrovacesa Mail,Metrovacesa,NaN,10058,2025-06-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_1026-OP_11_100_20250512H_13_38,02_METROVACESA_02,METROVACESA 02,NaN,1026,Metrovacesa Mail,Metrovacesa,NaN,10058,2025-06-02,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,101S_1026-OP_11_100_20250514H_09_16,03_METROVACESA_03,METROVACESA 03,NaN,1026,Metrovacesa Mail,Metrovacesa,NaN,10058,2025-06-03,...,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN
4,101S_1026-OP_11_98_20250516H_13_41,04_METROVACESA_04,METROVACESA 04,NaN,1026,Metrovacesa Mail,Metrovacesa,NaN,10058,2025-06-05,...,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN


**Limpieza de la columna empresa**

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfemail['EMPRESA'] = dfemail['EMPRESA'].str.lower().str.strip()

#dfemail['EMPRESA'] = (dfemail['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Reemplazar las empresas mal escritas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'

}

# Aplica los reemplazos
dfemail['EMPRESA'] = dfemail['EMPRESA'].replace(reemplazos)

**Armar la hoja resumen**

In [ ]:
# Asegurar que la columna se llama 'empresa'
dfemail.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dfemail['tipo_contacto'] = 'Mail/formulario'
dfemail['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dfemail = dfemail.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dfemail = dfemail[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dfemail

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Mail/formulario,Evaluada,22
1,adeslas,Mail/formulario,Evaluada,7
2,adt,Mail/formulario,Evaluada,10
3,aeda homes,Mail/formulario,Evaluada,5
4,aegon salud,Mail/formulario,Evaluada,1
...,...,...,...,...
78,viatris,Mail/formulario,Evaluada,12
79,volotea,Mail/formulario,Evaluada,14
80,whirlpool,Mail/formulario,Evaluada,8
81,xiaomi,Mail/formulario,Evaluada,1


# Web

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfwebE = pd.read_excel(file_path, sheet_name='Web')


dfwebE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. FACILIDAD.COND.ES SENCILLO UTILIZAR LOS MENÚS DE NAVEGACIÓN.,C13. FACILIDAD.COND.EN TODA LA NAVEGACIÓN WEB HAY PUBLICIDAD INNECESARIA.,C13. FACILIDAD.COND.NO SE MANTIENEN LOS MISMOS CRITERIOS DE ESTILO EN TODAS LAS PÁGINA FACILITANDO SU USO.,C13. FACILIDAD.COND.NO ES POSIBLE ENCONTRAR CONTENIDOS DE AYUDA SIN DIFICULTAD.,C13. FACILIDAD.COND.NO ES SENCILLO UTILIZAR LOS MENÚS DE NAVEGACIÓN.,C13. FACILIDAD.OBS,C14. DIVERSIDAD FUNCIONAL,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB SÍ TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB NO TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.OBS
0,101S_1057-OP_11_98_20250617H_11_10MD$2,01_SECTOR ALARM_09,Sector Alarm 09,NaN,1057,Sector Alarm Web,Sector Alarm,NaN,10059,2025-06-17,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
1,101S_1057-OP_11_98_20250617H_11_25MD$2,02_SECTOR ALARM_11,Sector Alarm 11,NaN,1057,Sector Alarm Web,Sector Alarm,NaN,10059,2025-06-17,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
2,101S_1057-OP_11_98_20250617H_11_32,03_SECTOR ALARM_13,Sector Alarm 06,NaN,1057,Sector Alarm Web,Sector Alarm,NaN,10059,2025-06-17,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
3,101S_1057-OP_11_98_20250617H_11_41,04_SECTOR ALARM_15,Sector Alarm 15,NaN,1057,Sector Alarm Web,Sector Alarm,NaN,10059,2025-06-17,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN
4,101S_1057-OP_11_98_20250617H_11_53,05_SECTOR ALARM_03,Sector Alarm 03,NaN,1057,Sector Alarm Web,Sector Alarm,NaN,10059,2025-06-17,...,0,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN


**Limpiar la columna empresa**

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfwebE['EMPRESA'] = dfwebE['EMPRESA'].str.lower().str.strip()

#dfwebE['EMPRESA'] = (dfwebE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Reemplazar los nombres mal escritos de empresas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfwebE['EMPRESA'] = dfwebE['EMPRESA'].replace(reemplazos)

**Crear hoja resumen**

In [ ]:
# Asegurar que la columna se llama 'empresa'
dfwebE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Agregar columnas fijas antes del agrupamiento
dfwebE['tipo_contacto'] = 'Web'
dfwebE['tipo_de_acción'] = 'Evaluada'

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dfwebE = dfwebE.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dfwebE = dfwebE[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
dfwebE

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Web,Evaluada,4
1,adeslas,Web,Evaluada,2
2,aeda homes,Web,Evaluada,3
3,asisa,Web,Evaluada,2
4,asus,Web,Evaluada,2
5,aurgi,Web,Evaluada,2
6,avis,Web,Evaluada,2
7,beko,Web,Evaluada,1
8,bosch,Web,Evaluada,1
9,bosch talleres,Web,Evaluada,4


# Facebook y Twitter

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredesE = pd.read_excel(file_path, sheet_name='Redes')


dfredesE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C14. HABILIDAD.COND.NO EXISTE OPCIÓN DE ADJUNTAR MATERIAL MULTIMEDIA PARA COMPLEMENTAR LA CONSULTA.,C14. HABILIDAD.OBS,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR),C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C15. SOLUCIÓN AL PRIMER CONTACTO (FCR).OBS,C7. FORMATO DE LA RESPUESTA.COND.LA RESPUESTA OCUPA MENOS DE 1 TWEET/6 LÍNEAS DE MD EN TWITTER.,C7. FORMATO DE LA RESPUESTA.COND.LA RESPUESTA OCUPA MÁS DE 1 TWEET/6 LÍNEAS DE MD EN TWITTER.,C11. DESPEDIDA.COND.EL ASESOR SÍ REALIZA UNA DESPEDIDA CORDIAL PARA DESPEDIR LA CONVERSACIÓN EN TWITTER.,C11. DESPEDIDA.COND.EL ASESOR NO REALIZA UNA DESPEDIDA CORDIAL PARA DESPEDIR LA CONVERSACIÓN EN TWITTER.
0,101S_1108-OP_11_98_20250513H_13_04,01_AURGI_01,Aurgi 01,NaN,1108,Aurgi Facebook,Aurgi,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,101S_1108-OP_11_98_20250514H_15_04,02_AURGI_02,Aurgi 02,NaN,1108,Aurgi Facebook,Aurgi,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_1109-OP_11_98_20250515H_09_39,03_AURGI_03,Aurgi 03,NaN,1109,Aurgi Twitter,Aurgi,NaN,10060,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0
3,101S_1109-OP_11_98_20250516H_09_20,04_AURGI_04,Aurgi 04,NaN,1109,Aurgi Twitter,Aurgi,NaN,10060,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0
4,101S_1113-OP_11_98_20250513H_16_00,01_AVIS_01,Avis 01,NaN,1113,Avis Facebook,Avis,NaN,10061,2025-05-19,...,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


**Reemplazar las empresas mal escritas**

In [ ]:
#Pasar a minúscula
dfredesE['EMPRESA'] = dfredesE['EMPRESA'].str.lower()

# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfredesE['EMPRESA'] = dfredesE['EMPRESA'].replace(reemplazos)

dfredesE['EMPRESA']

,EMPRESA
0,aurgi
1,aurgi
2,aurgi
3,aurgi
4,avis
5,avis
6,avis
7,avis
8,bosch
9,continental


**Tomar los canales correctos**

In [ ]:
# --- PASO 1: Asegurar minúsculas en columnas relevantes ---
dfredesE['SERVICIO'] = dfredesE['SERVICIO'].str.lower().str.lower()
#dfredesE['EMPRESA'] = (dfredesE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

# --- PASO 2: Definir palabras clave válidas y reemplazos ---
servicios_validos = [
    'twitter','facebook', 'canal social media facebook', 'social media facebook'
]

reemplazos_servicio = {
    'teléfono': 'Telefono',
    'llamadas': 'Telefono',
    'telefónico': 'Telefono',
    'canal telefónico': 'Telefono',
    'canal web':'Web',
    'canal social media facebook':'Facebook',
    'social media facebook': 'Facebook',
    'mail/formulario':'Mail/formulario',
    'web': 'Web',
    'facebook':'Facebook',
    'twitter':'Twitter',
    'chat web': 'Chat web',
    'whatsapp': 'WhatsApp',
    'webs':'Web'
}

# --- PASO 3: Función para limpiar SERVICIO ---
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos_servicio.get(palabra, palabra)
    return None

# --- PASO 4: Aplicar limpieza servicio ---
dfredesE['SERVICIO'] = dfredesE['SERVICIO'].apply(limpiar_servicio)

print(dfredesE['SERVICIO'])

0     Facebook
1     Facebook
2      Twitter
3      Twitter
4     Facebook
5     Facebook
6     Facebook
7     Facebook
8      Twitter
9     Facebook
10    Facebook
11    Facebook
12    Facebook
13    Facebook
14    Facebook
15    Facebook
16     Twitter
17    Facebook
18    Facebook
19    Facebook
20    Facebook
21    Facebook
22    Facebook
23    Facebook
24    Facebook
25    Facebook
26    Facebook
27    Facebook
28    Facebook
29    Facebook
30    Facebook
31    Facebook
32    Facebook
33    Facebook
34    Facebook
35    Facebook
36    Facebook
37    Facebook
38    Facebook
Name: SERVICIO, dtype: object


**Crear hoja resumen para FACEBOOK**

In [ ]:
# Filtramos solo "enviado" por canal
df_facebookE = dfredesE[(dfredesE['SERVICIO'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebookE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['SERVICIO']

# Agrupar y contar
df_facebookE = df_facebookE.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebookE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebookE.head()

📘 Facebook:


/tmp/ipython-input-52-19667561.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Evaluada'
/tmp/ipython-input-52-19667561.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['SERVICIO']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,asus,Facebook,Evaluada,2
1,aurgi,Facebook,Evaluada,2
2,avis,Facebook,Evaluada,4
3,bosch talleres,Facebook,Evaluada,4
4,bridgestone,Facebook,Evaluada,4


**TWITTER**

In [ ]:
# Filtramos solo "enviado" por canal
df_twitterE = dfredesE[(dfredesE['SERVICIO'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitterE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['SERVICIO']

# Agrupar y contar
df_twitterE = df_twitterE.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitterE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Twitter:")
df_twitterE.head()

📘 Twitter:


/tmp/ipython-input-53-4206790448.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_de_acción'] = 'Evaluada'
/tmp/ipython-input-53-4206790448.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['tipo_contacto'] = df_temp['SERVICIO']


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,aurgi,Twitter,Evaluada,2
1,bosch,Twitter,Evaluada,1
2,dia fidelización,Twitter,Evaluada,1


# Chat y Whatsapp

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluadas.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchatwebE = pd.read_excel(file_path, sheet_name='Chat web')


dfchatwebE.head()

,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. FACILIDAD.COND.NO ES SENCILLO UTILIZAR LOS MENÚS DE NAVEGACIÓN.,C13. FACILIDAD.OBS,C14. DIVERSIDAD FUNCIONAL,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB SÍ TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB NO TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL.,C14. DIVERSIDAD FUNCIONAL.OBS,C14. DIVERSIDAD FUNCIONAL.1,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB SÍ TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL..1,C14. DIVERSIDAD FUNCIONAL.COND.LA PÁGINA WEB NO TIENE OPCIÓN DE ACCESO PARA PERSONAS CON DISCAPACIDAD VISUAL..1,C14. DIVERSIDAD FUNCIONAL.OBS
0,101S_1124-OP_11_98_20250513H_16_50,01_BOSCH_01,Bosch 01,NaN,1124,Bosch Chat Web,Bosch,NaN,10062,2025-05-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101S_552-OP_11_98_20250513H_17_27,01_EDENRED_01,EDENRED Usuario 01,NaN,552,Edenred Usuario Chat Virtual,Edenred Usuarios,NaN,10062,2025-06-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101S_553-OP_11_98_20250513H_12_15,01_ACER_01,Acer 1,NaN,553,Acer Chat Virtual,Acer,NaN,10062,2025-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101S_673-OP_11_98_20250513H_17_22,01_DINAHOSTING_01,Dinahosting 01,NaN,673,Dinahosting Chat virtual,Dinahosting,NaN,10062,2025-05-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101S_677-OP_11_98_20250513H_16_39,01_ASUS_01,ASUS 01,NaN,677,ASUS Chat Virtual,ASUS,NaN,10062,2025-05-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Limpiar columna empresa**

In [ ]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfchatwebE['EMPRESA'] = dfchatwebE['EMPRESA'].str.lower().str.strip()

#dfchatwebE['EMPRESA'] = (dfchatwebE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Reemplazar empresas mal escritas**

In [ ]:
# Diccionario de reemplazo de empresas
reemplazos = {
    'aeda': 'aeda homes',
    'grupo agbar': 'agbar',
    'bosch car service': 'bosch talleres',
    'canal isabel ii': 'canal de isabel ii',
    'club dia': 'dia fidelización',
    'emma colchón': 'emma',
    'eurorepar': 'eurorepair',
    'fain': 'fain ascensores',
    'grundig': 'grunding',
    'lg electro': 'lg',
    'lg televisores': 'lg tv',
    'max colchón': 'maxcolchón',
    'nationale nederlanden': 'national nederlanden',
    'samsung telefonia': 'samsung',
    'samsung electro': 'samsung electrodomésticos',
    'samsung televisores': 'samsung tv',
    'sixt': 'sixty',
    'yves rocher': 'yvers rocher',
    '1 and 1 ionos': 'ionos',
    'emma colchones':'emma',
    'lg electrodomésticos': 'lg',
    'securitas direct':'securitas',
    'nationale nederlanden seguros de vida': 'national nederlanden',
    'edenred usuarios':'edenred',
    'aegon seguros de vida': 'aegon vida',
    'aegon seguros': 'aegon salud',
    'sector alarm': 'sector alarma',
    'samsung electrodomesticos':'samsung electrodomésticos',
    'samsung telefonía': 'samsung',
    'edenred clientes': 'edenred',
    'adt tyco': 'adt',
    'bosch car': 'bosch talleres',
    'yvers rochers': 'yvers rocher',
    'club caprabo': 'caprabo fidelización',
    'herts':'hertz',
    'caprabo distribución certamen': 'caprabo',
    'caprabo club certamen': 'caprabo fidelización',
    'bruenau':'bruneau',
    'whirpool':'whirlpool',
    'ritual':'rituals',
    'sector alarm potencial':'sector alarma',
    'general optica':'general óptica',
    'national netherlanden':'national nederlanden',
    'phillips':'philips'
}

# Aplica los reemplazos
dfchatwebE['EMPRESA'] = dfchatwebE['EMPRESA'].replace(reemplazos)

dfchatwebE['EMPRESA']
print(dfchatwebE[dfchatwebE['EMPRESA'] == 'canal de isabel ii'])

Empty DataFrame
Columns: [ID, IDEVENTCLIENTE, EQUIPO, CIP, ID SERVICIO, SERVICIO, EMPRESA, DISPOSITIVO, MEDIDOR, FECHA EVALUACIÓN, HORAEVALUACION, VALORACIÓN, IDOPERADOR, AGENTE, IDEVALUADOR, EVALUADOR, IDESPECIALISTA, RESPUESTA, DURACIÓN, FECHA, HORA, MOTIVO, ALCANCE, CATEGORIA1, CATEGORIA2, CATEGORIA3, CATEGORIA4, GRABACIÓN, ARCHIVO_ADJUNTO, ARCHIVO_ADJUNTO2, ARCHIVO_ADJUNTO3, ARCHIVO_ADJUNTO4, ARCHIVO_ADJUNTO5, C1. VISIBILIDAD, C1. VISIBILIDAD.COND.SE NECESITAN 4 O MENOS CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.COND.SE NECESITAN ENTRE 5 Y 6 CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.COND.SE NECESITAN 7 O MÁS CLICKS PARA ACCEDER AL CHAT WEB., C1. VISIBILIDAD.OBS, C2. TIEMPO DE ESPERA, C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE EN MENOS DE 30 SEGUNDOS., C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE ENTRE 30 SEGUNDOS Y 1 MINUTO 30 SEGUNDOS., C2. TIEMPO DE ESPERA.COND.EL ASESOR ATIENDE AL CLIENTE ENTRE 1 MINUTO Y 30 SEGUNDOS Y 3 MINUTOS., C2. TIEMPO 

**Tomar los canales válidos**

In [ ]:
# --- PASO 1: Asegurar minúsculas en columnas relevantes ---
dfchatwebE['SERVICIO'] = dfchatwebE['SERVICIO'].str.lower()
dfchatwebE['EMPRESA'] = (dfchatwebE['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

# --- PASO 2: Definir palabras clave válidas y reemplazos ---
servicios_validos = [
    'twitter','facebook', 'canal social media facebook', 'social media facebook', 'chat virtual', 'chat',  'whatsapp', 'chat web'
]

reemplazos_servicio = {
    'teléfono': 'Telefono',
    'llamadas': 'Telefono',
    'telefónico': 'Telefono',
    'canal telefónico': 'Telefono',
    'canal web':'Web',
    'canal social media facebook':'Facebook',
    'social media facebook': 'Facebook',
    'mail/formulario':'Mail/formulario',
    'web': 'Web',
    'facebook':'Facebook',
    'twitter':'Twitter',
    'chat web': 'Chat',
    'whatsapp': 'WhatsApp',
    'webs':'Web',
    'chat virtual': 'Chat',
    'chat': 'Chat'
}

# --- PASO 3: Función para limpiar SERVICIO ---
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos_servicio.get(palabra, palabra)
    return None

# --- PASO 4: Aplicar limpieza servicio ---
dfchatwebE['SERVICIO'] = dfchatwebE['SERVICIO'].apply(limpiar_servicio)

print(dfchatwebE['SERVICIO'])

0     Chat
1     Chat
2     Chat
3     Chat
4     Chat
5     Chat
6     Chat
7     Chat
8     Chat
9     Chat
10    Chat
11    Chat
12    Chat
13    Chat
14    Chat
15    Chat
16    Chat
17    Chat
18    Chat
19    Chat
20    Chat
21    Chat
Name: SERVICIO, dtype: object


**Armar hoja resumen para CHAT**

In [ ]:
# Asegurar que la columna se llama 'empresa'
dfchatwebE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Filtramos solo "enviado" por canal
dfchatE = dfchatwebE[(dfchatwebE['SERVICIO'] == 'Chat')]

# Agregar columnas fijas
for df_temp in [dfchatwebE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['SERVICIO']

# Agrupar por empresa, tipo_contacto y tipo_de_acción y contar ocurrencias
dfchatE = dfchatwebE.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas
dfchatE = dfchatE[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# MostErar resultado
dfchatE


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Chat,Evaluada,4
1,asus,Chat,Evaluada,4
2,bosch,Chat,Evaluada,2
3,bruneau,Chat,Evaluada,4
4,dinahosting,Chat,Evaluada,4
5,edenred,Chat,Evaluada,4


**Armar hoja resumen para WHATSAPP**

In [ ]:
# Filtramos solo "enviado" por canal
df_whatE = dfchatwebE[(dfchatwebE['tipo_contacto'] == 'WhatsApp')]

# Agregar columnas fijas
for df_temp in [df_whatE]:
    df_temp['tipo_de_acción'] = 'Evaluada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_whatE = df_whatE.groupby(['empresa', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_whatE.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 whatsapp:")
df_whatE.head()

📘 whatsapp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


# **Agrupar por EVALUADAS**

In [ ]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_evaluada = pd.concat([dftelef, dfemail, df_facebookE, df_twitterE, dfchatE, dfwebE], ignore_index=True)
df_final_evaluada = df_final_evaluada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_evaluada)

           empresa tipo_contacto tipo_de_acción  cantidad
0             acer      Telefono       Evaluada        48
1          adeslas      Telefono       Evaluada         9
2              adt      Telefono       Evaluada        28
3       aeda homes      Telefono       Evaluada        35
4      aegon salud      Telefono       Evaluada        16
..             ...           ...            ...       ...
202           ikea           Web       Evaluada         6
203        ryanair           Web       Evaluada         2
204  sector alarma           Web       Evaluada         6
205        volotea           Web       Evaluada         6
206        vueling           Web       Evaluada         2

[207 rows x 4 columns]


In [ ]:
recuento_contacto = df_final_evaluada.groupby('tipo_contacto')['cantidad'].sum().reset_index()
print("Cantidad total por tipo de contacto:")
print(recuento_contacto)

total_general = df_final_evaluada['cantidad'].sum()
print(f"Total general de cantidad: {total_general}")

Cantidad total por tipo de contacto:
     tipo_contacto  cantidad
0             Chat        22
1         Facebook        35
2  Mail/formulario       594
3         Telefono      1914
4          Twitter         4
5              Web        64
Total general de cantidad: 2633


In [ ]:
# Guardar el DataFrame modificado como archivo Excel
df_final_evaluada.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Registro.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ---------------------------------------------------------------------------------------------------------------------------------

# **Crear hoja Recuento por TIPO DE CONTACTO**

In [ ]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Resumen Planificación.xlsx'

# Cargar el archivo Excel
df2 = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df2.head()

,Empresa,Categoría,Inicio,Fin,BBDD,Observaciones BBDD,Compra,Fecha Informe,Telefono,Mail/formulario,Web,Twitter,Facebook,Chat,WhatsApp
0,Acer,Informática de Consumo,2025-05-05,2025-06-27,Sí,Hemos recibido número de series tanto pregunta...,No,NaN,135,55,6,6.0,6.0,12.0,0.0
1,Adeslas,Seguros de Salud,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,2,0.0,0.0,0.0,0.0
2,ADT,Sistemas de Seguridad,2025-06-02,2025-06-27,NaN,Es importante respetar el timing. Actualizar BBDD,NaN,NaN,135,55,6,12.0,0.0,12.0,0.0
3,Aeda Homes,Promotoras,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,68,27,3,0.0,0.0,0.0,0.0
4,Aegon Salud,Seguros de Salud,2025-05-19,2025-07-11,NaN,Hay BBDD y escenarios estrictos,NaN,NaN,135,35,6,0.0,0.0,0.0,0.0


**Unir los dos excel, obteniendo evaluada, realizada y total**

In [ ]:
# --- Paso 1: Unir evaluadas y realizadas ---
df_combined = pd.concat([df_final_evaluada, df_final_realizada], ignore_index=True)

df_combined

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Evaluada,48
1,adeslas,Telefono,Evaluada,9
2,adt,Telefono,Evaluada,28
3,aeda homes,Telefono,Evaluada,35
4,aegon salud,Telefono,Evaluada,16
...,...,...,...,...
442,bsm,Web,Realizada,3
443,canal de isabel ii,Web,Realizada,4
444,cottet,Web,Realizada,6
445,ikea,Web,Realizada,4


In [ ]:
import pandas as pd

# --- Normalización mínima: solo minúsculas y espacios ---
df_combined['empresa'] = df_combined['empresa'].str.lower().str.strip()
df_combined['tipo_contacto'] = df_combined['tipo_contacto'].str.strip()

df2['Empresa'] = df2['Empresa'].str.lower().str.strip()
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']

# --- Paso 1: Pivotear df_combined por empresa y canal ---
df_pivot = df_combined.pivot_table(
    index=['empresa', 'tipo_contacto'],
    columns='tipo_de_acción',
    values='cantidad',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Asegurarse de que columnas Evaluada y Realizada estén presentes
for col in ['Evaluada', 'Realizada']:
    if col not in df_pivot.columns:
        df_pivot[col] = 0

# Ordenar columnas
df_pivot = df_pivot[['empresa', 'tipo_contacto', 'Evaluada', 'Realizada']]

# --- Paso 2: Pasar df2 a formato largo ---
df2_melt = df2.melt(
    id_vars=['Empresa', 'Inicio', 'Fin'],
    value_vars=canales,
    var_name='tipo_contacto',
    value_name='Total'
)

df2_melt.rename(columns={'Empresa': 'empresa'}, inplace=True)
df2_melt['tipo_contacto'] = df2_melt['tipo_contacto'].str.strip()

# --- Paso 3: Unir los datos sin alterar nombres originales ---
df_merged = pd.merge(
    df2_melt,
    df_pivot,
    on=['empresa', 'tipo_contacto'],
    how='left'
)

# Rellenar NaN solo en columnas numéricas
df_merged[['Evaluada', 'Realizada', 'Total']] = df_merged[['Evaluada', 'Realizada', 'Total']].fillna(0)

# --- Paso 4: Pivotear a formato final (una fila por empresa con columnas por canal) ---
df_final = df_merged.pivot_table(
    index=['empresa', 'Inicio', 'Fin'],
    columns='tipo_contacto',
    values=['Evaluada', 'Realizada', 'Total'],
    aggfunc='sum',
    fill_value=0
)

# Aplanar columnas multiíndice
df_final.columns = [f"{canal}_{accion}" for accion, canal in df_final.columns]
df_final = df_final.reset_index()

# --- Paso 5: Unir con todas las empresas de df2 para no perder ninguna ---
df_base_empresas = df2[['Empresa', 'Inicio', 'Fin']].copy()
df_base_empresas.rename(columns={'Empresa': 'empresa'}, inplace=True)
df_base_empresas['empresa'] = df_base_empresas['empresa'].str.lower().str.strip()

# Unión externa para mantener empresas sin datos de acciones
df_final_completo = pd.merge(
    df_base_empresas,
    df_final,
    on=['empresa', 'Inicio', 'Fin'],
    how='left'
)

# Rellenar posibles NaN por empresas sin datos de acciones
df_final_completo = df_final_completo.fillna(0)

# --- Paso 6: Verificación de consistencia ---
empresas_originales = set(df2['Empresa'].str.lower().str.strip())
empresas_final = set(df_final_completo['empresa'])
empresas_faltantes = empresas_originales - empresas_final

if empresas_faltantes:
    print(f"⚠️ Empresas faltantes en el resultado final: {sorted(empresas_faltantes)}")
else:
    print("✅ Todas las empresas de df2 están presentes en df_final.")

# Resultado final
df_final = df_final_completo

df_final



✅ Todas las empresas de df2 están presentes en df_final.


,empresa,Inicio,Fin,Chat_Evaluada,Facebook_Evaluada,Mail/formulario_Evaluada,Telefono_Evaluada,Twitter_Evaluada,Web_Evaluada,WhatsApp_Evaluada,...,Twitter_Realizada,Web_Realizada,WhatsApp_Realizada,Chat_Total,Facebook_Total,Mail/formulario_Total,Telefono_Total,Twitter_Total,Web_Total,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,4.0,0.0,22.0,48.0,0.0,4.0,0.0,...,0.0,4.0,0.0,12.0,6.0,55.0,135.0,6.0,6.0,0.0
1,adeslas,2025-05-05,2025-06-27,0.0,0.0,7.0,9.0,0.0,2.0,0.0,...,0.0,2.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
2,adt,2025-06-02,2025-06-27,0.0,0.0,10.0,28.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,0.0,55.0,135.0,12.0,6.0,0.0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,5.0,35.0,0.0,3.0,0.0,...,0.0,3.0,0.0,0.0,0.0,27.0,68.0,0.0,3.0,0.0
4,aegon salud,2025-05-19,2025-07-11,0.0,0.0,1.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,35.0,135.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,volotea,2025-06-02,2025-06-27,0.0,0.0,14.0,10.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,55.0,135.0,0.0,6.0,0.0
99,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,30.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
100,whirlpool,2025-05-05,2025-06-27,0.0,0.0,8.0,17.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,14.0,34.0,2.0,2.0,0.0
101,xiaomi,2025-05-05,2025-06-27,0.0,0.0,1.0,17.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,19.0,45.0,4.0,2.0,2.0


In [ ]:
#Empresas sin fecha de inicio y fin
#print("📘 Empresas sin fecha de inicio y fin, a reemplazar a mano:")
#display(df_final_sin_fecha[['empresa'] + [col for col in df_final_sin_fecha.columns if col in ['Inicio', 'Fin']]])


**Ordenar columnas para que esten juntos por canal**

In [ ]:
# Paso final: ordenar columnas agrupadas por canal
orden_final = ['empresa', 'Inicio', 'Fin']
tipos = ['Realizada', 'Evaluada', 'Total']
for canal in canales:
    for tipo in tipos:
        col = f"{canal}_{tipo}"
        if col in df_final.columns:
            orden_final.append(col)

# Reordenar el DataFrame
df_final = df_final[orden_final]

df_final

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Facebook_Total,Twitter_Realizada,Twitter_Evaluada,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,76.0,48.0,135.0,26.0,22.0,55.0,4.0,...,6.0,0.0,0.0,6.0,4.0,4.0,12.0,0.0,0.0,0.0
1,adeslas,2025-05-05,2025-06-27,22.0,9.0,45.0,12.0,7.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,adt,2025-06-02,2025-06-27,69.0,28.0,135.0,28.0,10.0,55.0,0.0,...,0.0,0.0,0.0,12.0,0.0,0.0,12.0,0.0,0.0,0.0
3,aeda homes,2025-05-05,2025-06-27,39.0,35.0,68.0,15.0,5.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon salud,2025-05-19,2025-07-11,49.0,16.0,135.0,8.0,1.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,volotea,2025-06-02,2025-06-27,38.0,10.0,135.0,35.0,14.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,vueling,2025-06-02,2025-06-27,44.0,30.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,whirlpool,2025-05-05,2025-06-27,23.0,17.0,34.0,9.0,8.0,14.0,0.0,...,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
101,xiaomi,2025-05-05,2025-06-27,25.0,17.0,45.0,10.0,1.0,19.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,2.0,0.0,0.0,2.0


**--Generar una copia del df_final hasta aquí para luego trabajar el acumulativo--**

In [ ]:
# Crear una copia profunda del DataFrame (independiente de la original) para trabajar con el acumulativo a posteriori
#df_copy = df_final.copy()

#print("Copia modificada")
#print(df_copy.head())


**Obtener Total_general, total_evaluadas y total_realizadas**

In [ ]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_final.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_final.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_final.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
df_final['Total_general'] = df_final[cols_total].sum(axis=1)

df_final.head()

/tmp/ipython-input-69-1799761933.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
/tmp/ipython-input-69-1799761933.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
/tmp/ipython-input-69-1799761933.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general
0,acer,2025-05-05,2025-06-27,76.0,48.0,135.0,26.0,22.0,55.0,4.0,...,6.0,4.0,4.0,12.0,0.0,0.0,0.0,110.0,78.0,220.0
1,adeslas,2025-05-05,2025-06-27,22.0,9.0,45.0,12.0,7.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,18.0,65.0
2,adt,2025-06-02,2025-06-27,69.0,28.0,135.0,28.0,10.0,55.0,0.0,...,12.0,0.0,0.0,12.0,0.0,0.0,0.0,97.0,38.0,220.0
3,aeda homes,2025-05-05,2025-06-27,39.0,35.0,68.0,15.0,5.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,43.0,98.0
4,aegon salud,2025-05-19,2025-07-11,49.0,16.0,135.0,8.0,1.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,17.0,176.0


**Obtener los dias laborables restantes**

In [ ]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_final['Fin'] = pd.to_datetime(df_final['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_final['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_final.head()


/tmp/ipython-input-70-525242225.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Fin'] = pd.to_datetime(df_final['Fin'])
/tmp/ipython-input-70-525242225.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,76.0,48.0,135.0,26.0,22.0,55.0,4.0,...,4.0,4.0,12.0,0.0,0.0,0.0,110.0,78.0,220.0,5
1,adeslas,2025-05-05,2025-06-27,22.0,9.0,45.0,12.0,7.0,18.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,36.0,18.0,65.0,5
2,adt,2025-06-02,2025-06-27,69.0,28.0,135.0,28.0,10.0,55.0,0.0,...,0.0,0.0,12.0,0.0,0.0,0.0,97.0,38.0,220.0,5
3,aeda homes,2025-05-05,2025-06-27,39.0,35.0,68.0,15.0,5.0,27.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,57.0,43.0,98.0,5
4,aegon salud,2025-05-19,2025-07-11,49.0,16.0,135.0,8.0,1.0,35.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,57.0,17.0,176.0,15


**Obtener columnas por evaluar y por realizar según el total general**

In [ ]:
df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']

df_final.head()

/tmp/ipython-input-71-2731595869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
/tmp/ipython-input-71-2731595869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar
0,acer,2025-05-05,2025-06-27,76.0,48.0,135.0,26.0,22.0,55.0,4.0,...,12.0,0.0,0.0,0.0,110.0,78.0,220.0,5,142.0,110.0
1,adeslas,2025-05-05,2025-06-27,22.0,9.0,45.0,12.0,7.0,18.0,2.0,...,0.0,0.0,0.0,0.0,36.0,18.0,65.0,5,47.0,29.0
2,adt,2025-06-02,2025-06-27,69.0,28.0,135.0,28.0,10.0,55.0,0.0,...,12.0,0.0,0.0,0.0,97.0,38.0,220.0,5,182.0,123.0
3,aeda homes,2025-05-05,2025-06-27,39.0,35.0,68.0,15.0,5.0,27.0,3.0,...,0.0,0.0,0.0,0.0,57.0,43.0,98.0,5,55.0,41.0
4,aegon salud,2025-05-19,2025-07-11,49.0,16.0,135.0,8.0,1.0,35.0,0.0,...,0.0,0.0,0.0,0.0,57.0,17.0,176.0,15,159.0,119.0


**Calcular la cantidad de a realizar y evaluar según los días laborables restantes**

In [ ]:
# Asegurarse de evitar división por cero o días negativos
df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
df_final['A_evaluar_por_dia'] = ((df_final['Total_general'] - df_final['Total_evaluadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)

df_final.head()

/tmp/ipython-input-72-2181712680.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)
/tmp/ipython-input-72-2181712680.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
/tmp/ipython-input-72-2181712680.py:6: SettingWithCopyWarning: 
A value is trying to be set on 

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,WhatsApp_Evaluada,WhatsApp_Total,Total_realizadas,Total_evaluadas,Total_general,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,76.0,48.0,135.0,26.0,22.0,55.0,4.0,...,0.0,0.0,110.0,78.0,220.0,5,142.0,110.0,22,28
1,adeslas,2025-05-05,2025-06-27,22.0,9.0,45.0,12.0,7.0,18.0,2.0,...,0.0,0.0,36.0,18.0,65.0,5,47.0,29.0,6,9
2,adt,2025-06-02,2025-06-27,69.0,28.0,135.0,28.0,10.0,55.0,0.0,...,0.0,0.0,97.0,38.0,220.0,5,182.0,123.0,25,36
3,aeda homes,2025-05-05,2025-06-27,39.0,35.0,68.0,15.0,5.0,27.0,3.0,...,0.0,0.0,57.0,43.0,98.0,5,55.0,41.0,8,11
4,aegon salud,2025-05-19,2025-07-11,49.0,16.0,135.0,8.0,1.0,35.0,0.0,...,0.0,0.0,57.0,17.0,176.0,15,159.0,119.0,8,11


**Extraer el archivo final**

In [ ]:
# Guardar el DataFrame modificado como archivo Excel
df_final.to_excel('Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ----------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------------------------

# Cargar un archivo del histórico distinto al último creado.

**Cuidado**, el bloque "Generar acumulativo..." debe ejecutarse manualmente punto a punto para que no tome el último  archivo creado del histórico sino este que estamos escogiendo manualmente.

In [ ]:
import os
import glob
import pandas as pd

# Ruta base donde están los archivos históricos
ruta_base = "/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico"

# Buscar todos los archivos del historial
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

# Mostrar lista con índice
print("📁 Archivos históricos disponibles:")
for i, archivo in enumerate(archivos):
    print(f"{i+1}. {os.path.basename(archivo)}")

# Elegir índice del archivo a cargar
indice = int(input("\n👉 Ingresá el número del archivo que querés cargar: ")) - 1

# Verificación
if 0 <= indice < len(archivos):
    archivo_seleccionado = archivos[indice]
    df_final_actualizado = pd.read_excel(archivo_seleccionado)
    print(f"\n✅ Archivo cargado: {archivo_seleccionado}")
else:
    print("❌ Índice fuera de rango.")


📁 Archivos históricos disponibles:
1. Acumulativo_Recuento_historico_2025-05-13_1.xlsx
2. Acumulativo_Recuento_historico_2025-05-13_2.xlsx
3. Acumulativo_Recuento_historico_2025-05-13_3.xlsx
4. Acumulativo_Recuento_historico_2025-05-13_4.xlsx
5. Acumulativo_Recuento_historico_2025-05-14_1.xlsx
6. Acumulativo_Recuento_historico_2025-05-14_2.xlsx


KeyboardInterrupt: Interrupted by user

# -----------------------------------------------------------------------------------------------------

# **Generar acumulativo para ver progreso del RECUENTO**

**Generar histórico acumaltivo**

**Solo ejecutar una vez!!**
Toma los valores de realizada y evaluada para ir actualizando los resultados.

**Cargar acumulativo histórico**

In [ ]:
import os
import glob
import pandas as pd
from datetime import datetime

# Buscar el último archivo guardado
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

if archivos:
    archivo_historico = archivos[-1]
    df_historico = pd.read_excel(archivo_historico)
    print(f"✅ Último histórico cargado desde: {archivo_historico}")
else:
    # Crear histórico vacío con las columnas necesarias
    columnas_clave = ['empresa', 'Inicio', 'Fin']
    columnas_actualizar = [col for col in df_final.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]
    df_historico = pd.DataFrame(columns=columnas_clave + columnas_actualizar)
    print("⚠️ No hay acumulativos anteriores. Se parte de vacío.")

⚠️ No hay acumulativos anteriores. Se parte de vacío.


In [ ]:
# Limpiar espacios en nombres de columnas (importante antes del melt)
df2.columns = df2.columns.str.strip()

# También asegurarse que los valores en la columna 'Empresa' no tengan espacios
df2['Empresa'] = df2['Empresa'].str.strip()

# Verificamos que los nombres estén limpios y coincidan con 'canales'
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']


In [ ]:
# Filtrar columnas a actualizar
columnas_actualizar = [col for col in df_historico.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]

# Asegurarse de que los índices clave sean iguales antes de sumar
df_historico['empresa'] = df_historico['empresa'].str.strip().str.lower()
df_final['empresa'] = df_final['empresa'].str.strip().str.lower()

# Definir columnas clave
columnas_clave = ['empresa', 'Inicio', 'Fin']

# Hacemos merge para acumular
df_actualizado = pd.merge(
    df_historico,
    df_final[columnas_clave + columnas_actualizar],
    on=columnas_clave,
    how='outer',
    suffixes=('', '_nuevo')
)

# Sumar columnas *_Realizada y *_Evaluada
for col in columnas_actualizar:
    col_nuevo = f"{col}_nuevo"
    if col_nuevo in df_actualizado.columns:
        df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
        df_actualizado.drop(columns=col_nuevo, inplace=True)

# Seleccionar solo columnas *_realizada, *_evaluada, *_total y claves
columnas_filtradas = columnas_clave + [col for col in df_actualizado.columns if col.lower().endswith(('_realizada', '_evaluada', '_total'))]

df_actualizado = df_actualizado[columnas_filtradas]

df_actualizado

<ipython-input-104-754d7becf17a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['empresa'] = df_final['empresa'].str.strip().str.lower()
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
<ipython-input-104-754d7becf17a>:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Obtener total general, evaluadas y realizadas**

In [ ]:
# Identificamos columnas por tipo
cols_realizada = [col for col in df_actualizado.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_actualizado.columns if col.endswith('_Evaluada')]
cols_total = [col for col in df_actualizado.columns if col.endswith('_Total')]

# Creamos las columnas de suma total
df_actualizado['Total_realizadas'] = df_actualizado[cols_realizada].sum(axis=1)
df_actualizado['Total_evaluadas'] = df_actualizado[cols_evaluada].sum(axis=1)
df_actualizado['Total_general'] = df_actualizado[cols_total].sum(axis=1)

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,Facebook_Evaluada,Twitter_Realizada,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Calcular días restantes**

In [ ]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_actualizado['Fin'] = pd.to_datetime(df_actualizado['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_actualizado['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_actualizado['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29


**Obtener columnas por evaluar y realizar**

In [ ]:
df_actualizado['Por_realizar'] = df_actualizado['Total_general'] - df_actualizado['Total_realizadas']
df_actualizado['Por_evaluar'] = df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,Twitter_Evaluada,Chat web_Realizada,Chat web_Evaluada,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,-3.0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**A realizar y a evaluar según los días restantes**

In [ ]:
# Asegurarse de evitar división por cero o días negativos
df_actualizado['Dias_laborables_restantes'] = df_actualizado['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_actualizado['A_realizar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_realizadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)
df_actualizado['A_evaluar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)

df_actualizado

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Mail/formulario_Realizada,Mail/formulario_Evaluada,Web_Realizada,Web_Evaluada,Facebook_Realizada,...,WhatsApp_Realizada,WhatsApp_Evaluada,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,27.0,8.0,9.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,40.0,8.0,-40.0,-8.0,29,-1,0
1,adeslas,2025-05-05,2025-06-27,7.0,2.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,12.0,2.0,-12.0,-2.0,29,0,0
2,aeda homes,2025-05-05,2025-06-27,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,-3.0,29,0,0
3,aegon salud,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
4,aegon vida,2025-05-12,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,volotea,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
90,vueling,2025-06-02,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,0,0
91,whirlpool,2025-05-05,2025-06-27,7.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,-9.0,0.0,29,0,0
92,xiaomi,2025-05-05,2025-06-27,6.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,1.0,-8.0,-1.0,29,0,0


**Almacenar en Drive el histórico que alimentará al siguiente**

In [ ]:
fecha = datetime.now().strftime("%Y-%m-%d")
contador = len(glob.glob(os.path.join(ruta_base, f"Acumulativo_Recuento_historico_{fecha}_*.xlsx")))
nombre_archivo = f"Acumulativo_Recuento_historico_{fecha}_{contador + 1}.xlsx"
ruta_guardado = os.path.join(ruta_base, nombre_archivo)

df_actualizado.to_excel(ruta_guardado, index=False)

print(f"✅ Histórico actualizado y guardado en: {ruta_guardado}")

✅ Histórico actualizado y guardado en: /content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico/Acumulativo_Recuento_historico_2025-05-16_1.xlsx


**Descargar excel**

In [ ]:
# Guardar el DataFrame modificado como archivo Excel
df_actualizado.to_excel('Acumulativo Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Acumulativo Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ------------------------------------------------------------------------------------------------------------------